https://neo4j.com/labs/genai-ecosystem/langchain/

In [6]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "H3110_W0r1d"

graph = Neo4jGraph() # apoc issues : https://github.com/langchain-ai/langchain/issues/12901

In [7]:
from dotenv import load_dotenv

load_dotenv()

In [8]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-4-0125-preview")

llm_transformer = LLMGraphTransformer(llm=llm)

Failed to write data to connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))


In [9]:
from langchain_core.documents import Document

text = """
Marie Curie, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris.
"""
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Marie Curie', type='Person'), Node(id='Polish', type='Nationality'), Node(id='Naturalised-French', type='Nationality'), Node(id='Physicist', type='Occupation'), Node(id='Chemist', type='Occupation'), Node(id='Radioactivity', type='Field_of_study'), Node(id='Nobel Prize', type='Award'), Node(id='Pierre Curie', type='Person'), Node(id='University Of Paris', type='Organization')]
Relationships:[Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Polish', type='Nationality'), type='NATIONALITY'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Naturalised-French', type='Nationality'), type='NATIONALITY'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Physicist', type='Occupation'), type='OCCUPATION'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Chemist', type='Occupation'), type='OCCUPATION'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id

In [10]:
graph.add_graph_documents(graph_documents)

In [11]:
graph_documents

[GraphDocument(nodes=[Node(id='Marie Curie', type='Person'), Node(id='Polish', type='Nationality'), Node(id='Naturalised-French', type='Nationality'), Node(id='Physicist', type='Occupation'), Node(id='Chemist', type='Occupation'), Node(id='Radioactivity', type='Field_of_study'), Node(id='Nobel Prize', type='Award'), Node(id='Pierre Curie', type='Person'), Node(id='University Of Paris', type='Organization')], relationships=[Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Polish', type='Nationality'), type='NATIONALITY'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Naturalised-French', type='Nationality'), type='NATIONALITY'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Physicist', type='Occupation'), type='OCCUPATION'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Chemist', type='Occupation'), type='OCCUPATION'), Relationship(source=Node(id='Marie Curie', type='Person')

In [17]:
graph.refresh_schema()

In [18]:
graph.get_schema

'Node properties are the following:\nPerson {id: STRING},Nationality {id: STRING},Occupation {id: STRING},Field_of_study {id: STRING},Award {id: STRING},Organization {id: STRING}\nRelationship properties are the following:\n\nThe relationships are the following:\n(:Person)-[:PROFESSOR]->(:Organization),(:Person)-[:NATIONALITY]->(:Nationality),(:Person)-[:OCCUPATION]->(:Occupation),(:Person)-[:FIELD_OF_STUDY]->(:Field_of_study),(:Person)-[:AWARD_WON]->(:Award)'

In [22]:
from langchain.chains import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

chain.run("What did Marie Curie study?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {id: "Marie Curie"})-[:FIELD_OF_STUDY]->(f:Field_of_study)
RETURN f.id
Full Context:
[{'f.id': 'Radioactivity'}]

> Finished chain.


'Marie Curie studied Radioactivity.'